In [1]:
import requests
import lxml
from bs4 import BeautifulSoup as bSoup
import pandas as pd

In [2]:
url = 'https://www.amazon.co.uk/gp/bestsellers/computers/430594031/ref=pd_zg_hrsr_computers?fbclid=IwAR2UA1wEfy8AraOyhbecXBgdvxIn6Ki9DikwZRbhmEdhspFp3wpfGO0G_8o'

In [92]:
test = getSoup(url)

In [3]:
def getSoup(url):
    response = requests.get(url)
    responseParsed = bSoup(response.text, 'lxml')
    return responseParsed

In [4]:
def getElementSet(mainSoup):
    products = mainSoup.find_all('span', class_='aok-inline-block zg-item')
    return products

In [5]:
def cleanText(string):
    string = string.replace('\n',"")
    string = string.replace('\r',"").strip()
    return string

In [9]:
def getElement (parent, tag, class_):
    if parent.find(tag, class_=class_) == None:
        return (None)
    else:
        return cleanText(parent.find(tag, class_=class_).text)

In [89]:
def getTotalPages(mainSoup):
    pagination = mainSoup.find('ul', class_='a-pagination')
    if pagination.find('li', class_='a-last') == None:
        return (None)
    else:
        lastPage = pagination.find('li', class_='a-last')
        totalPages = lastPage.a['href'][lastPage.a['href'].find('pg'):len(lastPage.a['href'])]
        linkStruct = lastPage.a['href'][0:lastPage.a['href'].find('pg')-1]
        numberPages = int(totalPages[totalPages.find('=')+1:len(totalPages)])
        return linkStruct, numberPages

In [96]:
def makeLinkCollection(mainSoup):
    linkCollection = []
    if getTotalPages(mainSoup) == None:
        return None
    else:
        for i in range(1,getTotalPages(mainSoup)[1]+1):
            linkCollection.append(getTotalPages(mainSoup)[0]+'&pg='+str(i))
        return(linkCollection)

In [97]:
links = makeLinkCollection(test)

In [98]:
print(links)

['https://www.amazon.co.uk/Best-Sellers-Computers-Accessories-Computer-Microphones/zgbs/computers/430594031?_encoding=UTF8&pg=1', 'https://www.amazon.co.uk/Best-Sellers-Computers-Accessories-Computer-Microphones/zgbs/computers/430594031?_encoding=UTF8&pg=2']


In [84]:
def productParsing (product):
    Product = {}
    name = getElement(product,'div', class_='p13n-sc-truncate')
    price = getElement(product,'span', class_='p13n-sc-price') 
    stars = getElement(product,'span', class_='a-icon-alt')
    Product = {"Name":name,"Price":price,"Stars":stars}
    return Product

In [87]:
collection = []
x = 0
for product in products:
    collection.append(productParsing(product))
    x = x+1

In [91]:
dataframe = pd.DataFrame (collection)